In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(777)
device

device(type='cuda')

## Task1


1. 생성할 문장 데이터

In [ ]:
sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

### 1. 데이터 전처리

2. 문자 집합 만들기

In [ ]:
world_set = sorted(list(set(sentence)))  # 글자 단위로 쪼개기

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {word : index for index, word in enumerate(world_set)}

In [ ]:
print(vocab)

{' ': 0, "'": 1, ',': 2, '.': 3, 'B': 4, 'a': 5, 'b': 6, 'c': 7, 'd': 8, 'e': 9, 'f': 10, 'g': 11, 'h': 12, 'i': 13, 'k': 14, 'l': 15, 'm': 16, 'n': 17, 'o': 18, 'p': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'w': 24, 'y': 25}


3. 문자 집합 크기 확인

In [ ]:
vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

In [ ]:
hidden_size = vocab_size 
# 입력은 원-핫 벡터를 사용할 것이므로 입력의 크기는 문자 집합의 크기여야만 함
# hidden_size(은닉층의 크기)를 입력의 크기와 동일하게 줬는데, 다른 값을 줘도 무방
sequence_length = 10  # 샘플을 10개 단위로 끊어서 샘플을 만들 예정
learning_rate = 0.01

5. sequence 길이 단위 자르기

In [ ]:
# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequence

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1: i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [ ]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!
print(x_data[0])
print(y_data[0])

[4, 20, 13, 7, 14, 0, 24, 5, 15, 15]
[20, 13, 7, 14, 0, 24, 5, 15, 15, 21]


In [ ]:
len(x_data)

188

6. 입력 시퀀스에 대해 원핫인코딩 수행
7. 입력 데이터, 레이블데이터 텐서로 변환


    데이터 타입 + cpu/gpu 에 따라서 텐서 타입 다르게 : https://subinium.github.io/pytorch-Tensor-Variable/
    

In [ ]:
## 문제(4) : x_data를 원핫인코딩 
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

## 문제(5) : x_one_hot과 y_data를 텐서로 변환 
# torch.Tensor() = torch.tensor([])  
X = torch.cuda.FloatTensor(x_one_hot)
Y = torch.cuda.LongTensor(y_data) # "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Float'

8. 크기 확인

In [ ]:
print('훈련 데이터의 크기 : {}'.format(X.shape)) # (batch size, 문장 길이, 단어 벡터의 차원)
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


9. 원핫인코딩 결과 샘플 확인하기

In [ ]:
print(X[0])

tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

10. 레이블 데이터 샘플 확인하기

In [ ]:
print(Y[0])

tensor([20, 13,  7, 14,  0, 24,  5, 15, 15, 21], device='cuda:0')


### 2. 모델 구현하기

11. RNN 모델 구현 문자 단위 RNN

In [ ]:
## 문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)
### https://luminitworld.tistory.com/75

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_size, hidden_size, bias=True)
  
  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [ ]:
net = Net(vocab_size, hidden_size, 2) # 현재 hidden_size는 vocab_size와 같음

net.to(device) # gpu에 올리지 않으면 모델의 파라미터는 cpu, 데이터는 gpu에 얹어놓은 상태라는 내용의 .. 에러 띄움

Net(
  (rnn): RNN(26, 26, num_layers=2, batch_first=True)
  (fc): Linear(in_features=26, out_features=26, bias=True)
)

In [ ]:
device

device(type='cuda')

12. loss function
13. optimizer

In [ ]:
criterion = torch.nn.CrossEntropyLoss() 
optimizer = optim.Adam(net.parameters(), learning_rate)

14. 출력 크기 점검
    - 출력 크기 = 배치 차원, 시점(timesteps), 출력의 크기
    - 나중에 정확도 측정할 때는 모두 펼쳐서 계산하게 되는데, view를 사용하여 배치 차원과 시점 차원을 하나로 만든다

In [ ]:
outputs = net(X)      # nn.RNN()은 기본적으로 3차원 텐서를 입력받는다
print(outputs.shape)  # 배치 차원, 시점(timesteps), 출력의 크기 
print(outputs.view(-1, vocab_size).shape)

torch.Size([188, 10, 26])
torch.Size([1880, 26])


In [ ]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([188, 10])
torch.Size([1880])


15. Training 시작
16. 예측결과 확인

In [ ]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X) # (188, 10, 26) 크기의 텐서를 매 에포크마다 모델의 입력으로 사용
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #########  예측결과 확인 ####################

    results = outputs.argmax(dim=2) # result 텐서 크기는 (188,10) 
    predict_str = ""
    for j, result in enumerate(results): 
        if j == 0: # 처음(0)에는 예측 결과 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음부터는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

ygnnnnnnnnnnnnnrnnnnnnnnnnnrnnnnnnnnnnrnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnrnnnnnnnnnnnnrnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnrnnnnnnnnrnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn
nrnrnrnrrrrrrrnrrrrrrrrrrrrrnrrrrrrrrnrrrrrnrrnrrrnrrrrrrrrrrrrrrrrrrrrnrnrrrrrrrnrrnrnrrrrrrrrrrrrrrrrrrrnrrrrrrrrnrrrrrrnrrrrnrrrrrrrrrrrrrrrrnrnrrrrrrrnrrrrrrrrrrrrrnrrrrrrnrrrrrrrnrrrrrnrrrrrrn
r rr r rr  arr r rr  rrr rrr  rrrrrr  r rrr r  rrr r rrr r rrrrrrrrr  r rarr  arr    r rr  r rr r rrr rrr  rrr r r rr rrrr rrrrarrarrr rrrrrr  r rarr  arr r rr  r rrrrrrrr rrrrrrr   aarr rr r rrrr 
r           a             a        a           a              a                a                                                       a                a                             a       a      
r                                                                                                                                                                                                    
r         

In [ ]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to khow us how badly we want thinksm"

## Task2 - 한국어 텍스트



In [ ]:
sentence = ("그러나 한편 냉정히 생각해보면 큰 욕심 안 부리고 노력한 것만큼만 잘살아보겠다는 게 과연 보통 사람의 경지일까? "
            "보통 사람이란 좌절한 욕망을 한 장의 올림픽복권에 걸고 일주일 동안 행복하고 허황된 꿈을 꾸는 사람이 아닐까? "
            "보통 사람의 숨은 허욕이 없다면 주택복권이나 올림픽복권이 그렇게 큰 이익을 올릴 수는 없을 것이다. "
            "이 풍진세상風塵世上에서 노력한 만큼만 잘살기를 바라고 딴 욕심이 없다면 그건 보통 사람을 훨씬 넘은 성인의 경지이다.  "
            "그럼 진짜 보통 사람은 어디 있는 것일까? 과연 있기는 있는 것일까? 보통 사람이란 평균 점수처럼 어떤 집단을 대표하고 싶어 하는 가공의 숫자일 뿐, "
            "실지로 존재하는 것은 아닐지도 모른다."
            "- 박완서 '모래알만한 진실이라도'")

### 1. 데이터 전처리

2. 문자 집합 만들기

In [ ]:
world_set = sorted(list(set(sentence)))  
vocab = {word : index for index, word in enumerate(world_set)}

print(vocab)

{' ': 0, "'": 1, ',': 2, '-': 3, '.': 4, '?': 5, '上': 6, '世': 7, '塵': 8, '風': 9, '가': 10, '각': 11, '건': 12, '걸': 13, '것': 14, '게': 15, '겠': 16, '경': 17, '고': 18, '공': 19, '과': 20, '권': 21, '균': 22, '그': 23, '기': 24, '까': 25, '꾸': 26, '꿈': 27, '나': 28, '냉': 29, '넘': 30, '노': 31, '는': 32, '닐': 33, '다': 34, '단': 35, '대': 36, '도': 37, '동': 38, '된': 39, '디': 40, '딴': 41, '떤': 42, '라': 43, '란': 44, '람': 45, '래': 46, '러': 47, '럼': 48, '렇': 49, '력': 50, '로': 51, '른': 52, '를': 53, '리': 54, '릴': 55, '림': 56, '만': 57, '망': 58, '면': 59, '모': 60, '바': 61, '박': 62, '보': 63, '복': 64, '부': 65, '뿐': 66, '사': 67, '살': 68, '상': 69, '생': 70, '서': 71, '성': 72, '세': 73, '수': 74, '숨': 75, '숫': 76, '실': 77, '심': 78, '싶': 79, '씬': 80, '아': 81, '안': 82, '알': 83, '어': 84, '없': 85, '에': 86, '연': 87, '올': 88, '완': 89, '욕': 90, '은': 91, '을': 92, '의': 93, '이': 94, '익': 95, '인': 96, '일': 97, '있': 98, '자': 99, '잘': 100, '장': 101, '재': 102, '절': 103, '점': 104, '정': 105, '존': 106, '좌': 107, '주': 108, '지': 109, '진': 110,

3. 문자 집합 크기 확인

In [ ]:
vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 131


4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

In [ ]:
hidden_size = vocab_size 
sequence_length = 10 
learning_rate = 0.01

5. sequence 길이 단위 자르기

In [ ]:
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1: i+sequence_length+1]
  print(i, x_str, "->", y_str)

  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 그러나 한편 냉정히 -> 러나 한편 냉정히 
1 러나 한편 냉정히  -> 나 한편 냉정히 생
2 나 한편 냉정히 생 ->  한편 냉정히 생각
3  한편 냉정히 생각 -> 한편 냉정히 생각해
4 한편 냉정히 생각해 -> 편 냉정히 생각해보
5 편 냉정히 생각해보 ->  냉정히 생각해보면
6  냉정히 생각해보면 -> 냉정히 생각해보면 
7 냉정히 생각해보면  -> 정히 생각해보면 큰
8 정히 생각해보면 큰 -> 히 생각해보면 큰 
9 히 생각해보면 큰  ->  생각해보면 큰 욕
10  생각해보면 큰 욕 -> 생각해보면 큰 욕심
11 생각해보면 큰 욕심 -> 각해보면 큰 욕심 
12 각해보면 큰 욕심  -> 해보면 큰 욕심 안
13 해보면 큰 욕심 안 -> 보면 큰 욕심 안 
14 보면 큰 욕심 안  -> 면 큰 욕심 안 부
15 면 큰 욕심 안 부 ->  큰 욕심 안 부리
16  큰 욕심 안 부리 -> 큰 욕심 안 부리고
17 큰 욕심 안 부리고 ->  욕심 안 부리고 
18  욕심 안 부리고  -> 욕심 안 부리고 노
19 욕심 안 부리고 노 -> 심 안 부리고 노력
20 심 안 부리고 노력 ->  안 부리고 노력한
21  안 부리고 노력한 -> 안 부리고 노력한 
22 안 부리고 노력한  ->  부리고 노력한 것
23  부리고 노력한 것 -> 부리고 노력한 것만
24 부리고 노력한 것만 -> 리고 노력한 것만큼
25 리고 노력한 것만큼 -> 고 노력한 것만큼만
26 고 노력한 것만큼만 ->  노력한 것만큼만 
27  노력한 것만큼만  -> 노력한 것만큼만 잘
28 노력한 것만큼만 잘 -> 력한 것만큼만 잘살
29 력한 것만큼만 잘살 -> 한 것만큼만 잘살아
30 한 것만큼만 잘살아 ->  것만큼만 잘살아보
31  것만큼만 잘살아보 -> 것만큼만 잘살아보겠
32 것만큼만 잘살아보겠 -> 만큼만 잘살아보겠다
33 만큼만 잘살아보겠다 -> 큼만 잘살아보겠다는
34 큼만 잘살아보겠다는 -> 만 잘살아보겠다는 
35 만 잘살아보겠다는  ->  잘살아보겠다는 게
36

In [ ]:
print(x_data[0])
print(y_data[0])

[23, 47, 28, 0, 124, 118, 0, 29, 105, 130]
[47, 28, 0, 124, 118, 0, 29, 105, 130, 0]


6. 입력 시퀀스에 대해 원핫인코딩 수행
7. 입력 데이터, 레이블데이터 텐서로 변환


    데이터 타입 + cpu/gpu 에 따라서 텐서 타입 다르게 : https://subinium.github.io/pytorch-Tensor-Variable/
    

In [ ]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

X = torch.cuda.FloatTensor(x_one_hot)
Y = torch.cuda.LongTensor(y_data) 

8. 크기 확인

In [ ]:
print('훈련 데이터의 크기 : {}'.format(X.shape)) # (batch size, 문장 길이, 단어 벡터의 차원)
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([362, 10, 131])
레이블의 크기 : torch.Size([362, 10])


9. 원핫인코딩 결과 샘플 확인하기

In [ ]:
print(X[0])

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]], device='cuda:0')


10. 레이블 데이터 샘플 확인하기

In [ ]:
print(Y[0])

tensor([ 47,  28,   0, 124, 118,   0,  29, 105, 130,   0], device='cuda:0')


### 2. 모델 구현하기

11. RNN 모델 구현 문자 단위 RNN

In [ ]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_size, hidden_size, bias=True)
  
  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x



net = Net(vocab_size, hidden_size, 2) 
net.to(device) 

Net(
  (rnn): RNN(131, 131, num_layers=2, batch_first=True)
  (fc): Linear(in_features=131, out_features=131, bias=True)
)

12. loss function
13. optimizer

In [ ]:
criterion = torch.nn.CrossEntropyLoss() 
optimizer = optim.Adam(net.parameters(), learning_rate)

14. 출력 크기 점검
    - 출력 크기 = 배치 차원, 시점(timesteps), 출력의 크기
    - 나중에 정확도 측정할 때는 모두 펼쳐서 계산하게 되는데, view를 사용하여 배치 차원과 시점 차원을 하나로 만든다

In [ ]:
outputs = net(X)     
print(outputs.shape)  
print(outputs.view(-1, vocab_size).shape)

print(Y.shape)
print(Y.view(-1).shape)

torch.Size([362, 10, 131])
torch.Size([3620, 131])
torch.Size([362, 10])
torch.Size([3620])


15. Training 시작
16. 예측결과 확인

In [ ]:
for i in range(200):
    optimizer.zero_grad()
    outputs = net(X) 
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #########  예측결과 확인 ####################

    results = outputs.argmax(dim=2) 
    predict_str = ""
    for j, result in enumerate(results): 
        if j == 0: 
            predict_str += ''.join([world_set[t] for t in result])
        else: 
            predict_str += world_set[result[-1]]

    print(predict_str)

라동씬동라동동있동동세있라라있동동있동있있있있있있있동세있동라세있력세있라있있세있세동있세동동동라라있있있라있있동있있동동있라있있동라있동있세람있라동있동세동라동있있동있있있있세각세있동동있있동라세있세있있세있있있있동세있있동라있세라있동있세력라라동있있있라있있동동있동세있동있있있있동있동있동각있세라있있있세각세라있동동있세동있라세있있세있있세있있동있력동있있세있동있라세있세동동동있세있동라세있있있라있있라있동있있동세있동있있동있있있있동라동있있있동라있있동세있동있동있있동동동있있동동있있세있동동세있있있있있라있동있있력라있있세있있동라라동있라세동있세있동세있있동라라있있동라있동있세있동라있력라세있동있세있동세동있력세동동있동동있있세있동동동있있있동있있세있있세있있있동있세있있동있동있세동있동동있동세있세동동있라세동력라세있있세있동
                                                                                                                                                                                                                                                                                                                                                                                   
                                                                                                                                                                                                                                                                

In [ ]:
sentence

"그러나 한편 냉정히 생각해보면 큰 욕심 안 부리고 노력한 것만큼만 잘살아보겠다는 게 과연 보통 사람의 경지일까? 보통 사람이란 좌절한 욕망을 한 장의 올림픽복권에 걸고 일주일 동안 행복하고 허황된 꿈을 꾸는 사람이 아닐까? 보통 사람의 숨은 허욕이 없다면 주택복권이나 올림픽복권이 그렇게 큰 이익을 올릴 수는 없을 것이다. 이 풍진세상風塵世上에서 노력한 만큼만 잘살기를 바라고 딴 욕심이 없다면 그건 보통 사람을 훨씬 넘은 성인의 경지이다.  그럼 진짜 보통 사람은 어디 있는 것일까? 과연 있기는 있는 것일까? 보통 사람이란 평균 점수처럼 어떤 집단을 대표하고 싶어 하는 가공의 숫자일 뿐, 실지로 존재하는 것은 아닐지도 모른다.- 박완서 '모래알만한 진실이라도'"

In [ ]:
predict_str

"럼나 한편 냉정히 생각해보면 큰 욕심 안 부리고 노력한 것만큼만 잘살아보겠다는 게 과연 보통 사람의 경지일까? 보통 사람이란 평절한 욕망을 한 장의 올림픽복권에 걸고 일주일 동안 행복하고 허황된 꿈을 꾸는 사람이 아닐까? 보통 사람의 숨은 허욕이 없다면 주택복권이나 올림픽복권이 그렇게 큰 이익을 올릴 수는 없을 것이다. 이 풍진세상風塵世上에서 노력한 만큼만 잘살기를 바라고 딴 욕심이 없다면 그건 보통 사람을 훨씬 넘은 성인의 경지이다.  그럼 진짜 보통 사람은 어디 있는 것일까? 과연 있기는 있는 것일까? 보통 사람이란 평균 점수처럼 어떤 집단을 대표하고 싶어 하는 가공의 숫자일 뿐, 실지로 존재하는 것은 아닐지도 모른다.- 박완서 '모래알만한 진실이라도'"

전처리 하나도 안했는데 거의 똑같이 텍스트 생성됨 